In [1]:
import requests
import pandas as pd
import pymysql
import numpy as np
import urllib.parse as parse
from datetime import datetime

In [2]:
name_url = 'https://api.defistation.io/defiTvlList'
bnblockedList_data_url = 'https://api.defistation.io/bnblockedList/{defiName}?days=30'
tvl_data_list = 'https://api.defistation.io/chart/{defiName}?days=90'
token = "Basic OmMyNDVhNGEyLTYxZjgtMTFlYi1hZTkzLTAyNDJhYzEzMDAwMg=="

In [38]:
def get_data(url):
    response = requests.get(url, headers = {"Authorization":token})
    return response.json()

In [44]:
data = get_data(tvl_data_list.format(defiName="BXH"))
data

{'defiName': 'BXH', 'result': {}, 'price': {}}

In [5]:
defi_names = []
for defi_details in data:
    defi_names.append([defi_details['name'], defi_details['token']])
defi_names

[['pancake', 'CAKE'],
 ['Venus', 'XVS'],
 ['Alpaca Finance', 'ALPACA'],
 ['Ellipsis Finance', 'EPS'],
 ['Belt Finance', 'BELT'],
 ['MDEX', 'MDX'],
 ['Autofarm', 'AUTO'],
 ['PancakeBunny', 'BUNNY'],
 ['O3 Swap', 'O3'],
 ['Biswap', 'BSW'],
 ['Wault.Finance', 'WEX'],
 ['beefy.finance', 'BIFI'],
 ['ApeSwap', 'BANANA'],
 ['Bakery Swap', 'BAKE'],
 ['Rabbit Finance', 'RABBIT'],
 ['ACryptoS', 'ACS'],
 ['BTC Standard Hashrate Token', 'BTCST'],
 ['BunnyPark', 'BP'],
 ['Nerve Finance', 'NRV'],
 ['ForTube', 'FOR'],
 ['Jetfuel.Finance', 'FUEL'],
 ['dForce', 'DF'],
 ['Cream Finance', 'CREAM'],
 ['Alpha Homora', 'ALPHA'],
 ['Linear Finance', 'LINA'],
 ['SwampFinance', 'SWAMP'],
 ['Dopple Finance', 'DOP'],
 ['Bunicorn', None],
 ['ApeRocket', None],
 ['DeFireX', 'DFX'],
 ['Definix', None],
 ['Growing Farm', 'GROW'],
 ['Nominex', 'NMX'],
 ['bZx Protocol', 'BZRX'],
 ['TEN', 'TENFI'],
 ['Fleta Connect', 'CHERRY'],
 ['Goose Finance', 'EGG'],
 ['dFuture', 'DFT'],
 ['TreeDefi', 'TREE'],
 ['TimeWarp', None],


In [30]:
len(defi_names)

125

In [27]:
def convert_into_rows(data, key):
    if data and key in data:
        data = data[key]
        rows = []
        for key in data:
            date_key = datetime.utcfromtimestamp(int(key)).strftime('%Y-%m-%d')
            rows.append([date_key, data[key]])
        return rows
    else:
        print(f"{key} No data")
        return []

In [37]:
def assemble_into_df(defi_name_details, tvl_result, bnblocked):
    bnblocked_df = pd.DataFrame(convert_into_rows(bnblocked, 'result'), columns=['date', 'bnb_locked'])
    min_date = bnblocked_df.date.min()
    price_df = pd.DataFrame(convert_into_rows(bnblocked, 'price'), columns=['date', 'price'])
    mc_df = pd.DataFrame(convert_into_rows(bnblocked, 'marketCap'), columns=['date', 'mrk_cap'])
    holder_df = pd.DataFrame(convert_into_rows(bnblocked, 'holders'), columns=['date', 'holders'])
    tvl_df = pd.DataFrame(convert_into_rows(tvl_result, 'result'), columns=['date', 'tvl'])
    df = pd.merge(bnblocked_df, price_df, how='outer', left_on='date', right_on=['date'])
    df = pd.merge(df, mc_df, how='outer', left_on='date', right_on=['date'])
    df = pd.merge(df, holder_df, how='outer', left_on='date', right_on=['date'])
    df = pd.merge(df, tvl_df, how='outer', left_on='date', right_on=['date'])
    print(f"{df.shape}\t{df.date.min()}\t{df.date.max()}\t{defi_name_details}")
    df = df[df['date'] > min_date]
    df['name'] = defi_name_details[0]
    df['token'] = defi_name_details[1]
    df['date'] = pd.to_datetime(df['date'])
    return df.sort_values(['date']).reset_index(drop=True)
    

In [39]:
data_dfs = pd.DataFrame([])
for defi_name_details in defi_names:
    defiName = parse.quote(defi_name_details[0])
    bnblocked_data = get_data(bnblockedList_data_url.format(defiName=defiName))
    tvl_data = get_data(tvl_data_list.format(defiName=defiName))
    defi_name_data_df = assemble_into_df(defi_name_details, tvl_data, bnblocked_data)
    data_dfs = data_dfs.append(defi_name_data_df)


(90, 6)	2021-05-06	2021-08-03	['pancake', 'CAKE']
(90, 6)	2021-05-06	2021-08-03	['Venus', 'XVS']
(90, 6)	2021-05-06	2021-08-03	['Alpaca Finance', 'ALPACA']
(90, 6)	2021-05-06	2021-08-03	['Ellipsis Finance', 'EPS']
(90, 6)	2021-05-06	2021-08-03	['Belt Finance', 'BELT']
(90, 6)	2021-05-06	2021-08-03	['MDEX', 'MDX']
(90, 6)	2021-05-06	2021-08-03	['Autofarm', 'AUTO']
(90, 6)	2021-05-06	2021-08-03	['PancakeBunny', 'BUNNY']
(90, 6)	2021-05-06	2021-08-03	['O3 Swap', 'O3']
(90, 6)	2021-05-06	2021-08-03	['Biswap', 'BSW']
(90, 6)	2021-05-06	2021-08-03	['Wault.Finance', 'WEX']
(90, 6)	2021-05-06	2021-08-03	['beefy.finance', 'BIFI']
(90, 6)	2021-05-06	2021-08-03	['ApeSwap', 'BANANA']
(90, 6)	2021-05-06	2021-08-03	['Bakery Swap', 'BAKE']
(90, 6)	2021-05-06	2021-08-03	['Rabbit Finance', 'RABBIT']
(90, 6)	2021-05-06	2021-08-03	['ACryptoS', 'ACS']
(90, 6)	2021-05-06	2021-08-03	['BTC Standard Hashrate Token', 'BTCST']
(90, 6)	2021-05-06	2021-08-03	['BunnyPark', 'BP']
(90, 6)	2021-05-06	2021-08-03	['Ner

In [34]:
data_dfs

,date,bnb_locked,price,mrk_cap,holders,tvl,name,token
0,2021-07-06,0.0,15.88,3051134114.120146,418750.0,7692687015,pancake,CAKE
1,2021-07-07,0.0,14.54,2804462120.871468,426279.0,8037653335,pancake,CAKE
2,2021-07-08,0.0,13.98,2699989769.559308,434240.0,7519925893,pancake,CAKE
3,2021-07-09,0.0,14.69,2846492661.625878,438473.0,7336155516,pancake,CAKE
4,2021-07-10,0.0,15.12,2940705407.343217,440742.0,7569592750,pancake,CAKE
...,...,...,...,...,...,...,...,...
24,2021-07-30,0.0,NaN,NaN,NaN,9572,UnagiSwap Finance,None
25,2021-07-31,0.0,NaN,NaN,NaN,9572,UnagiSwap Finance,None
26,2021-08-01,0.0,NaN,NaN,NaN,9572,UnagiSwap Finance,None
27,2021-08-02,0.0,NaN,NaN,NaN,9572,UnagiSwap Finance,None


In [ ]:
def __connect__():
    db_user = 'coin'
    db_password = 'dragonstone#123'
    db_name = 'master_data'
    db_host = 'localhost'
    return pymysql.connect(
        unix_socket=None,
        host=db_host,
        user=db_user,
        password=db_password,
        db=db_name,
        charset='latin1',
        cursorclass=pymysql.cursors.DictCursor)


def get_cursor(connection):
    if connection is None:
        connection = __connect__()
    connection.ping(reconnect=True)
    return connection.cursor()


def insert_defi_neg_div_score(df):
    df = df.astype(object).where(pd.notnull(df), None)
    connection = __connect__()
    cursor = get_cursor(connection)
    for i, row in df.iterrows():
        try:
            dt = row["date"].timestamp()
            symbol = row['token'] if row['token'] else ' '
            print(row['date'], row['name'], row['token'], row['bnb_locked'], row['price'], row['mrk_cap'], row['holders'], row['tvl'])
            sql = """
            INSERT INTO `defistation_data` (`timestamp_dt`, `name`, `token`, `price`, `bnb_locked`, `mrk_cap`, `holders`, `tvl`) VALUES (FROM_UNIXTIME(%s), %s, %s, %s, %s, %s, %s, %s)
            ON DUPLICATE KEY UPDATE
            `token` = VALUES(token),
            `price` = VALUES(price),
            `bnb_locked` = VALUES(bnb_locked),
            `mrk_cap` = VALUES(mrk_cap),
            `holders` = VALUES(holders),
            `tvl` = VALUES(tvl)
            """
            cursor.execute(sql, (dt , row['name'], symbol, row['bnb_locked'], row['price'], row['mrk_cap'], row['holders'], row['tvl']))
        except Exception as e:
            print(e)
    connection.commit()
    cursor.close()
    connection.close()

In [ ]:
insert_defi_neg_div_score(df)

In [36]:
len(data_dfs.name.unique())

81

In [ ]:
df.date.max()

In [23]:
df.to_csv('defistation_data.csv')

NameError: name 'df' is not defined